In [1]:
# Import librairies.
library(httr)
library(rjson)
library(dplyr)
library(RMySQL)
library(dbConnect)
library(odbc)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: DBI
Loading required package: gWidgets

Attaching package: ‘gWidgets’

The following object is masked from ‘package:dplyr’:

    id



In [2]:
# Get credentials from .json.
creds <- fromJSON(file="./creds.json")
user <- creds[["user"]]
password <- creds[["password"]]
host <- creds[["url"]]
port <- creds[["port"]]
dbname <- creds[["database"]]

In [3]:
# Connecting to DB.
db = dbConnect(RMySQL::MySQL(), dbname=dbname, host=host, port=port, user=user, password=password)

In [4]:
# Reading and storing tables.
features <- dbReadTable(db, "features")
sampleSubmission <- dbReadTable(db, "sampleSubmission")
stores <- dbReadTable(db, "stores")
test <- dbReadTable(db, "test")
train <- dbReadTable(db, "train")

In [5]:
head(features)
head(train)
head(stores)

index,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NA,NA,NA,NA,NA,211.0964,8.106,0
1,1,2010-02-12,38.51,2.548,NA,NA,NA,NA,NA,211.2422,8.106,1
2,1,2010-02-19,39.93,2.514,NA,NA,NA,NA,NA,211.2891,8.106,0
3,1,2010-02-26,46.63,2.561,NA,NA,NA,NA,NA,211.3196,8.106,0
4,1,2010-03-05,46.50,2.625,NA,NA,NA,NA,NA,211.3501,8.106,0
5,1,2010-03-12,57.79,2.667,NA,NA,NA,NA,NA,211.3806,8.106,0


index,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,0
1,1,1,2010-02-12,46039.49,1
2,1,1,2010-02-19,41595.55,0
3,1,1,2010-02-26,19403.54,0
4,1,1,2010-03-05,21827.90,0
5,1,1,2010-03-12,21043.39,0


index,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875
5,6,A,202505


In [6]:
# Joining train tables
all_tables_train <- inner_join(train, features, by=c("Store", "Date", "IsHoliday"))
all_tables_train <- inner_join(all_tables_train, stores, by="Store")

# # Joining test tables
all_tables_test <- inner_join(test, features, by=c("Store", "Date", "IsHoliday"))
all_tables_test <- inner_join(all_tables_train, stores, by="Store")

write.csv(all_tables_train, "~/Documents/walmart/all_data_train.json")
write.csv(all_tables_test, "~/Documents/walmart/all_data_test.json")

## Cleaning the training data

In [7]:
str(all_tables_train)

'data.frame':	421570 obs. of  19 variables:
 $ index.x     : num  0 1 2 3 4 5 6 7 8 9 ...
 $ Store       : num  1 1 1 1 1 1 1 1 1 1 ...
 $ Dept        : num  1 1 1 1 1 1 1 1 1 1 ...
 $ Date        : chr  "2010-02-05" "2010-02-12" "2010-02-19" "2010-02-26" ...
 $ Weekly_Sales: num  24924 46039 41596 19404 21828 ...
 $ IsHoliday   : int  0 1 0 0 0 0 0 0 0 0 ...
 $ index.y     : num  0 1 2 3 4 5 6 7 8 9 ...
 $ Temperature : num  42.3 38.5 39.9 46.6 46.5 ...
 $ Fuel_Price  : num  2.57 2.55 2.51 2.56 2.62 ...
 $ MarkDown1   : num  NA NA NA NA NA NA NA NA NA NA ...
 $ MarkDown2   : num  NA NA NA NA NA NA NA NA NA NA ...
 $ MarkDown3   : num  NA NA NA NA NA NA NA NA NA NA ...
 $ MarkDown4   : num  NA NA NA NA NA NA NA NA NA NA ...
 $ MarkDown5   : num  NA NA NA NA NA NA NA NA NA NA ...
 $ CPI         : num  211 211 211 211 211 ...
 $ Unemployment: num  8.11 8.11 8.11 8.11 8.11 ...
 $ index       : num  0 0 0 0 0 0 0 0 0 0 ...
 $ Type        : chr  "A" "A" "A" "A" ...
 $ Size        : num  151

In [8]:
all_tables_train$Weekly_Sales<- ifelse(is.na(all_tables_train$Weekly_Sales), mean(all_tables_train$Weekly_Sales, na.rm=TRUE), all_tables_train$Weekly_Sales)
all_tables_train$IsHoliday<- ifelse(is.na(all_tables_train$IsHoliday), mean(all_tables_train$IsHoliday, na.rm=TRUE), all_tables_train$IsHoliday)
all_tables_train$Temperature <- ifelse(is.na(all_tables_train$Temperature), mean(all_tables_train$Temperature, na.rm=TRUE), all_tables_train$Temperature)
all_tables_train$Fuel_Price <- ifelse(is.na(all_tables_train$Fuel_Price), mean(all_tables_train$Fuel_Price, na.rm=TRUE), all_tables_train$Fuel_Price)
all_tables_train$CPI <- ifelse(is.na(all_tables_train$CPI), mean(all_tables_train$CPI, na.rm=TRUE), all_tables_train$CPI)
all_tables_train$Unemployment <- ifelse(is.na(all_tables_train$Unemployment), mean(all_tables_train$Unemployment, na.rm=TRUE), all_tables_train$Unemployment)
all_tables_train$Size<- ifelse(is.na(all_tables_train$Size), mean(all_tables_train$Size, na.rm=TRUE), all_tables_train$Size)

In [9]:
all_tables_train$MarkDown1 <- ifelse(is.na(all_tables_train$MarkDown1), mean(all_tables_train$MarkDown1, na.rm=TRUE), all_tables_train$MarkDown1)
all_tables_train$MarkDown2 <- ifelse(is.na(all_tables_train$MarkDown2), mean(all_tables_train$MarkDown2, na.rm=TRUE), all_tables_train$MarkDown2)
all_tables_train$MarkDown3 <- ifelse(is.na(all_tables_train$MarkDown3), mean(all_tables_train$MarkDown3, na.rm=TRUE), all_tables_train$MarkDown3)
all_tables_train$MarkDown4 <- ifelse(is.na(all_tables_train$MarkDown4), mean(all_tables_train$MarkDown4, na.rm=TRUE), all_tables_train$MarkDown4)
all_tables_train$MarkDown5 <- ifelse(is.na(all_tables_train$MarkDown5), mean(all_tables_train$MarkDown5, na.rm=TRUE), all_tables_train$MarkDown5)

In [10]:
cleaned_data <-  subset(all_tables_train, select=-c(index.x,index.y,index))

In [ ]:
write.csv(cleaned_data,"./clean.csv")